# Testbed initializaton

Read configuration

In [ ]:
from octoprobe.octoprobe import NTestRun
from octoprobe.util_dut_programmers import FirmwareDownloadSpec
from octoprobe.util_pytest import util_logging

from testbed import util_testbed
from testbed.constants import EnumFut, TentacleType

util_logging.init_logging()

fut = EnumFut.FUT_I2C

TESTBED = util_testbed.get_testbed()
ctx = NTestRun(testbed=TESTBED)

Select 3 tentacles - As there are two MCU tentacles connected, we specify one by its serial

In [ ]:
mcu = TESTBED.get_tentacle(
    tentacle_type=TentacleType.TENTACLE_MCU,
    # serial="e46340474b4e1831",   # pico
    serial="e46340474b174429",  # pyboard
)
daq = TESTBED.get_tentacle(tentacle_type=TentacleType.TENTACLE_DAQ_SALEAE)
potpourry = TESTBED.get_tentacle(tentacle_type=TentacleType.TENTACLE_DEVICE_POTPOURRY)

tentacles = [mcu, daq, potpourry]

# Tentacle Infra

Connect and powercycle

In [ ]:
# INFRA: Powercycle all tentacles
ctx.session_powercycle_tentacles()

# INFRA: Initialize tentacles
[t.infra.setup_infra(udev=None) for t in tentacles]

Communicate with the tentacle infra MCU

In [ ]:
mcu.infra.mcu_infra.get_micropython_version()

## Tentacle Relays

Close tentalces relays to allow I2C testing

In [ ]:
fut = EnumFut.FUT_I2C
for t in tentacles:
    t.set_relays_by_FUT(fut=fut, open_others=True)

Explicitely open/close the relays

In [ ]:
# INFRA: Set relays according to FUT\n",
mcu.infra.mcu_infra.relays(relays_close=[2, 7], relays_open=[1,2,3,4,5,6,7])

# Tentacle DUT

Flash the MCU soldered on tentacles DUT

**RPI_PICO2, v1.24**

In [ ]:
firmware_spec=FirmwareDownloadSpec.factory_json({
    "board_variant": "RPI_PICO2",
    "url": "https://micropython.org/resources/firmware/RPI_PICO2-20240809-v1.24.0-preview.201.g269a0e0e1.uf2",
    "micropython_version_text": "3.4.0; MicroPython v1.24.0-preview.201.g269a0e0e1 on 2024-08-09"
})
# TODO: Raise exception if board_variant does not equal
mcu.power.dut = True # TODO: This line should not be required
mcu.dut.mp_remote_close() # TODO: This line should not be required
mcu.flash_dut(udev_poller=ctx.udev_poller,firmware_spec=firmware_spec)

**RPI_PICO2, ...**

In [ ]:
firmware_spec=FirmwareDownloadSpec.factory_json({
    "board_variant": "RPI_PICO2",
    "url": "https://micropython.org/resources/firmware/RPI_PICO2-RISCV-20241016-v1.24.0-preview.449.g1b89c503d.uf2",
    "micropython_version_text": "3.4.0; MicroPython v1.24.0-preview.449.g1b89c503d on 2024-10-16"
})
mcu.power.dut = True # TODO: This line should not be required
mcu.dut.mp_remote_close() # TODO: This line should not be required
mcu.flash_dut(udev_poller=ctx.udev_poller,firmware_spec=firmware_spec)

**PYBV11, v1.23**

In [ ]:
firmware_spec=FirmwareDownloadSpec.factory_json({
    "board_variant": "PYBV11",
    "url": "https://micropython.org/resources/firmware/PYBV11-20240602-v1.23.0.dfu",
    "micropython_version_text": "3.4.0; MicroPython v1.23.0 on 2024-06-02"
})
mcu.power.dut = True # TODO: This line should not be required
mcu.dut.mp_remote_close() # TODO: This line should not be required
mcu.flash_dut(udev_poller=ctx.udev_poller,firmware_spec=firmware_spec)

Power on DUT MCU and wait for USB device to appear

In [ ]:
mcu.dut.mp_remote_close() # TODO: This line should not be required
mcu.power.dut = True # TODO: This line should not be required
mcu.dut_boot_and_init_mp_remote(udev=ctx.udev_poller)

Free serial port and return port name. Now we may connect using Thonny

In [ ]:
mcu.dut.mp_remote_close()

DUT MCU: Run some micropython code: blink red led

In [ ]:
mcu.dut.mp_remote.exec_raw("""
from pyb import Timer

tim = Timer(1, freq=1000)
tim.freq(2.0)
tim.callback(lambda serial_portt: pyb.LED(1).toggle())
""")
